In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = "./data/2ND YEAR ROLL LIST.xlsx"
aktu = 120
ours = 100
session = "2021"

# Making groups data

In [3]:
def preprocessData(path):
    """Takes the MultiSection file path and return the combine sorted dataframe and its length"""
    FILE = pd.ExcelFile(path)
    dfs = [list((iD, pd.read_excel(path, sheet_name=i))) for iD,i in enumerate(FILE.sheet_names) if "LATERAL" not in i]
    for i in range(len(dfs)):
        dfs[i][1]=dfs[i][1][2:]
        dfs[i][1].columns = dfs[i][1].loc[2, dfs[i][1].columns]
        for j in dfs[i][1].columns:
            if j not in ["Roll No.", "S.No."]:
                del dfs[i][1][j]
        dfs[i][1]=dfs[i][1][1:]
        dfs[i][1]=dfs[i][1].set_index("S.No.")
        dfs[i][1]["Batch"]=np.nan

    df = pd.concat([i[1] for i in dfs])
    df = df.sort_values("Roll No.")
    df = df.astype({"Roll No.":str})
    len_df = len(df)
    df.index = [i+1 for i in range(len_df)]
    return df, len_df

In [4]:
def isAlloted(SUB:str, LAB:str, Date:str, GROUP:str="OLD")->int:
    """
    Checks weather lab is alloted to a subject on a date or not
    returns
    0: Not Alloted
    1: ALL Alloted(Make Schdule)
    2: Lab Alloted
    3: Group Alloted
    4: Group performed
    """
    alloted = []
    global batch_no
    for i in next(os.walk("./data/Alloted/"))[2]:
        sub = i.split()[0]
        lab = "Lab "+ i.split()[2]
        date = i.split()[3][:-5]
        allotedGroups = list(pd.read_excel(f"./data/Alloted/{i}")["Batch"].value_counts().to_dict().keys())
        alloted.append((sub, lab, date, allotedGroups))
    f = 0
    for i in alloted:
        if SUB==i[0] and LAB==i[1] and Date==i[2] and GROUP in i[3]: # Make Schedule
            f = 1
            break
        elif SUB!=i[0] and LAB==i[1] and Date==i[2] and (GROUP not in i[3] or GROUP in i[3]): # Change Lab
            f = 2
            break
        elif SUB!=i[0] and LAB!=i[1] and Date==i[2] and GROUP in i[3]: # Group is alloted already
            f = 3
            break
        elif SUB==i[0] and (LAB==i[1] or LAB!=i[1]) and (Date!=i[2] or Date==i[2]) and GROUP in i[3]:
            f=4
            break
    if f==1:
        grps_path = f"./data/Alloted/{i[0]} {i[1]} {i[2]}.xlsx"
        return (f, grps_path)
    return f

In [5]:
def display():
    alloted = []
    for i in next(os.walk("./data/Alloted/"))[2]:
        sub = i.split()[0]
        lab = "Lab "+ i.split()[2]
        date = i.split()[3][:-5]
        allotedGroups = list(pd.read_excel(f"./data/Alloted/{i}")["Batch"].value_counts().to_dict().keys())
        alloted.append((sub, lab, date, allotedGroups))
    return alloted

In [6]:
def display_dir():
    alloted = []
    for i in next(os.walk("./data/Alloted/"))[2]:
        sub = i.split()[0]
        lab = "Lab "+ i.split()[2]
        date = i.split()[3][:-5]
        allotedGroups = list(pd.read_excel(f"./data/Alloted/{i}")["Batch"].value_counts().to_dict().keys())
        alloted.append((sub, lab, date, allotedGroups))
    a = []
    b=[]
    for i in range(len(alloted)):
        for j in range(4):
            a.append((alloted[i][0], alloted[i][1], alloted[i][2], np.nan, np.nan, slots[j]))
            b.append((alloted[i][3][j]))
    a = pd.MultiIndex.from_tuples(a)
    b = pd.DataFrame(b, index=a, columns=["Groups"])
    return b

In [7]:
def display_dir():
    global alloted
    alloted = []
    for i in next(os.walk("./data/Alloted/"))[2]:
        sub = i.split()[0]
        lab = "Lab "+ i.split()[2]
        date = i.split()[3][:-5]
        allotedGroups = list(pd.read_excel(f"./data/Alloted/{i}")["Batch"].value_counts().to_dict().keys())
        alloted.append((sub, lab, date, allotedGroups))
    a = []
    b=[]
    for i in range(len(alloted)):
        for j in range(len(alloted[i][3])):
            a.append((alloted[i][0], alloted[i][1], alloted[i][2], np.nan, np.nan, slots[j]))
            b.append((alloted[i][3][j]))
    a = pd.MultiIndex.from_tuples(a)
    b = pd.DataFrame(b, index=a, columns=["Groups"])
    return b

In [12]:
display()

[('COA', 'Lab 5', '20-02-2024', ['G1', 'G2', 'G3', 'G4'])]

In [13]:
289//30

9

In [8]:
def updateBatch():
    with open("./batch.txt", "r+") as f:
        global batch_no, ic
        if batch_no !=0 and ic!=1:
            f.write(f"{batch_no}, {ic}")
            return
        batch_no, ic = f.read().split()
        batch_no = int(batch_no[0])
        ic = int(ic)

In [103]:
batch_no, ic = 8, 241
updateBatch()

In [10]:
slots = {
    0:"9:00-10:30",
    1:"10:30-12:00",
    2:"1:00-2:30",
    3:"2:30-4:00"
}

In [111]:
display_dir().to_excel(
    "./data/output.xlsx",
    sheet_name="ouput",
    index_label=["Subject", "Lab", "Date", "Internal Faculty", "External Faculty", "Time"])

In [125]:
display_dir()

IndexError: list index out of range

In [34]:
def makeGroups(df, len_df, date, sub, lab, aktu=1, initalize=0):
    """
    Make groups and returns the allocation details
    0: Groups are make successfully (Also return grouped df)
    1: All performed
    2: Lab Alloted
    3: Group Alloted
    """
    total_1_day = 120
    if aktu==0:
        total_1_day=100
    global batch_no,ic
    if initalize==1:
        if os.path.exists(f"./data/Alloted/{sub} {lab} {date}.xlsx"):
            os.remove(f"./data/Alloted/{sub} {lab} {date}.xlsx")
        batch_no, ic=0, 1
    else:
        batch_no = max(int(i[1]) for i in list(pd.read_excel(f"./data/Alloted/{sub} {lab} {date}.xlsx")["Batch"].value_counts().to_dict().keys()))
        ic = len(pd.read_excel(f"./data/Alloted/{sub} {lab} {date}.xlsx"))+1
    batchCounter = batch_no
    indexCounter= ic
    if batchCounter<(len_df//30-1) and isAlloted(sub, lab, date, f"G{batch_no+1}") in [0, 3, 4]:
        if isAlloted(sub, lab, date, f"G{batch_no+1}") in [3, 4]:
            while isAlloted(sub, lab, date, f"G{batchCounter+1}") in [3, 4]:
                print("Batch:", batchCounter+1, "Performed", indexCounter)
                batchCounter +=1
                indexCounter += 30
            if batchCounter+1==len_df//30+1:
                return 1
        for i in range(1,total_1_day//30+1):
            for j in range(indexCounter, indexCounter+30,1):
                batch_no=batchCounter+i
                allocation = isAlloted(sub, lab, date, f"G{batch_no}")
                if allocation==0:
                    df.loc[j,"Batch"] = f"G{batch_no}"
            indexCounter += 30
        ic=indexCounter
        df = df.dropna()
        df.to_excel(f"./data/Alloted/{sub} {lab} {date}.xlsx", index=False)
        return 0, df
    elif (len_df/30!=0) and isAlloted(sub, lab, date, "Other Dept.") in [0, 3, 4]:
        if isAlloted(sub, lab, date, f"G{batchCounter+1}") in [3, 4]:
            return 1
        for i in range(1,total_1_day//30+1):
            for j in range(indexCounter, indexCounter+30,1):
                batch_no=batchCounter+i
                allocation = isAlloted(sub, lab, date, f"G{batch_no}")
                if allocation==0:
                    df.loc[j,"Batch"] = "Other Dept."
            indexCounter += 30
        ic=indexCounter
        df = df.dropna()
        df.to_excel(f"./data/Alloted/{sub} {lab} {date}.xlsx", index=False)
        return 0, df
    else:
        return isAlloted(sub, lab, date, f"G{batch_no+1}")


Now, we are capable of creating the groups.  
with all itteration it increment automatically

In [132]:
df, len_df = preprocessData(path)
makeGroups(df, len_df, "5-2-2024", "Python", "Lab 9" ,aktu=1, initalize=1)

Batch: 1 Performed 1
Batch: 2 Performed 31
Batch: 3 Performed 61
Batch: 4 Performed 91


(0,
 2         Roll No. Batch
 121  2001430100123    G5
 122  2001430100124    G5
 123  2001430100125    G5
 124  2001430100127    G5
 125  2001430100128    G5
 ..             ...   ...
 236  2001430100246    G8
 237  2001430100247    G8
 238  2001430100248    G8
 239  2001430100249    G8
 240  2001430100250    G8
 
 [120 rows x 2 columns])

In [128]:
batch_no=0
ic=1

In [37]:
isAlloted("Python", "Lab 9", "5-2-2024", f"G{batch_no-1}")

NameError: name 'batch_no' is not defined

In [129]:
display_dir()

Groups
KCS751A Lab 5 5-2-24            NaN NaN 9:00-10:30      G1
                                        10:30-12:00     G2
                                        1:00-2:30       G3
                                        2:30-4:00       G4
        Lab Y Yet_To_be_Decided NaN NaN 9:00-10:30      G5
                                        10:30-12:00     G6
                                        1:00-2:30       G7
                                        2:30-4:00       G8

In [38]:
def chkoutliner(roll):
    if f"{session[-2:]}014301" not in str(roll):
        return True
    return False

def joinoutliner(otr, grp):
    "Join the outliners to the group"
    a = ""
    for i in otr:
        a+=i+", "
    a=a[:-2]+"\n"
    result = a+ f"{min(grp)} - {max(grp)}"
    return result

In [135]:
def rangeGroups(uploaded, sub, lab, date, len_df):
    global min, max
    ranges = []
    df = pd.read_excel(f"./data/Alloted/{sub} {lab} {date}.xlsx")
    df.index = [i+1 for i in range(len(df))]
    df = df.astype({"Roll No.": str})
    try:
        G1 = df[df["Batch"]=="G1"]
        pre = list(G1[G1["Roll No."].apply(chkoutliner)].to_dict()["Roll No."].values())
        G1 = set(G1["Roll No."].to_list()) - set(pre)
        G1 = list(G1)
        G1.sort()
        ranges.append(["G1", joinoutliner(pre, G1), len(G1)+len(pre)])
    except Exception as e:
        print(e.__class__)
    for i in df["Batch"].value_counts().to_dict().keys():
        if i!="G1":
            exec(f"{i} = list(df[df['Batch']=='{i}'].to_dict()['Roll No.'].values())")
            Max, Min, Total = eval(f"max({i})"), eval(f"min({i})"), eval(f"len({i})")
            ranges.append([f'{i}', f"{Min} - {Max}", Total])
    df = pd.read_excel("./data/output.xlsx", sheet_name="Groups")
    total_performed = df["Total Students"].sum()
    if total_performed in [240, 220, 200] and len_df>total_performed:
        otr = list(uploaded[total_performed+1:])
        ranges.append(["Other Deptartment", f"{min(otr)} - {max(otr)}", len(otr)])
    df = pd.DataFrame(ranges, columns=["Group", "Roll No. Range", "Total Students"])
    return df

In [140]:
df = preprocessData(path)[0]

In [148]:
df = pd.read_excel("./data/output.xlsx", sheet_name="Groups")
df["Total Students"].sum()

254

Till now the decision of roll No. range is done

In [85]:
rangeGroups(
    date="08-02-2024",
    sub="KCS553",
    lab="Lab 6",
    initalize=1
)
# ).to_excel("./data/a.xlsx", index=False)

<class 'KeyError'>


,Group,Roll No. Range,Total Students
0,G5,2101430100135 - 2101430100165,30
1,G6,2101430100166 - 2101430100200,30
2,G7,2101430100201 - 2201430109018,25


In [15]:
slots = {
    0:"9:00-10:30",
    1:"10:30-12:00",
    2:"1:00-2:30",
    3:"2:30-4:00"
}

In [69]:
## Make scedule for single subject
def makeSchedule(sub, date, lab, initalize=0):
    global batch_no
    Batch_no = batch_no
    if initalize==1:
        Batch_no=1
    print(Batch_no, sub, lab, date, f"G{Batch_no}")
    df = pd.DataFrame([[np.nan for _ in range(4)] for _ in range(4)], columns=["Date", "Time", "Subject", "Group"], index=np.arange(1, 5))
    allocation = isAlloted(sub, lab, date, f"G{Batch_no}")
    print(allocation)
    if type(allocation)==type(()):
        grps_path = allocation[1]
        grps = list(pd.read_excel(grps_path)["Batch"].value_counts().to_dict().keys())
        print(grps)
        for i in range(1, 5):
            df.loc[i, "Date"] = date
            df.loc[i, "Time"] = slots[i-1]
            df.loc[i, "Subject"] = sub
            df.loc[i, "Group"] = grps[i-1]
        df["Internal Faculty"] = [np.nan]*4
        df["External Faculty"] = [np.nan]*4
    return df

In [70]:
makeSchedule("AI", "5-4-2024", "Lab 9", initalize=0)

8 AI Lab 9 5-4-2024 G8


(1, './data/Alloted/AI Lab 9 5-4-2024.xlsx')
['G5', 'G6', 'G7', 'G8']


,Date,Time,Subject,Group,Internal Faculty,External Faculty
1,5-4-2024,9:00-10:30,AI,G5,NaN,NaN
2,5-4-2024,10:30-12:00,AI,G6,NaN,NaN
3,5-4-2024,1:00-2:30,AI,G7,NaN,NaN
4,5-4-2024,2:30-4:00,AI,G8,NaN,NaN


In [206]:
def rangeGroups(sub, lab, date, len_df):
    global min, max
    ranges = []
    df = pd.read_excel(f"./data/Alloted/{sub} {lab} {date}.xlsx")
    df.index = [i+1 for i in range(len(df))]
    df = df.astype({"Roll No.": str})
    try:
        G1 = df[df["Batch"]=="G1"]
        pre = list(G1[G1["Roll No."].apply(chkoutliner)].to_dict()["Roll No."].values())
        G1 = set(G1["Roll No."].to_list()) - set(pre)
        G1 = list(G1)
        G1.sort()
        ranges.append(["G1", joinoutliner(pre, G1), len(G1)+len(pre)])
    except Exception as e:
        print(e.__class__)
    for i in df["Batch"].value_counts().to_dict().keys():
        if i!="G1":
            exec(f"{i} = list(df[df['Batch']=='{i}'].to_dict()['Roll No.'].values())")
            Max, Min, Total = eval(f"max({i})"), eval(f"min({i})"), eval(f"len({i})")
            ranges.append([f'{i}', f"{Min} - {Max}", Total])
    if int(i[1])>=len_df//30:
        ranges.pop()
        G = df[df["Batch"]==f"{i}"]
        # print(G.tail())
        lat = list(G[G['Roll No.'].apply(chkoutliner)].to_dict()["Roll No."].values())
        G = set(G["Roll No."].to_list()) - set(lat)
        G = list(G)
        G.sort()
        ranges.append([i, joinoutliner(lat, G), len(G)+len(lat)])
    df = pd.DataFrame(ranges, columns=["Group", "Roll No. Range", "Total Students"])
    return df


In [152]:
# updates groups
if os.path.exists("./data/groups.xlsx"):
    df1 = pd.read_excel("./data/groups.xlsx", sheet_name="Groups")
df2 = pd.read_excel("./data/output.xlsx", sheet_name="ouput")